I started with a fork of Radek's notebook based on the walkthrough of the fastai course - then followed the path Jeremy was outlining on the later walkthroughs.

At the end of this notebook, we will have submitted to the ["Paddy Doctor: Paddy Disease Classification" competition on Kaggle](https://www.kaggle.com/competitions/paddy-disease-classification/overview)!

Let us begin by downloading the data.  This is persisted - so not needed each time.

In [1]:
%%bash

exec bash
rm -rf data
mkdir data
apt install unzip


Reading package lists...
Building dependency tree...
Reading state information...
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 43 not upgraded.
Need to get 168 kB of archives.
After this operation, 567 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 unzip amd64 6.0-21ubuntu1.1 [168 kB]
Fetched 168 kB in 0s (2844 kB/s)
Selecting previously unselected package unzip.
(Reading database ... 44825 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-21ubuntu1.1_amd64.deb ...
Unpacking unzip (6.0-21ubuntu1.1) ...
Setting up unzip (6.0-21ubuntu1.1) ...
Processing triggers for mime-support (3.60ubuntu1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...




debconf: delaying package configuration, since apt-utils is not installed


In [2]:
pip install -U timm==0.6.2dev0

Note: you may need to restart the kernel to use updated packages.


In [3]:
!cd data && kaggle competitions download -c paddy-disease-classification && unzip -q paddy-disease-classification.zip

100%|█████████████████████████████████████▉| 1.02G/1.02G [00:47<00:00, 22.4MB/s]
100%|██████████████████████████████████████| 1.02G/1.02G [00:47<00:00, 22.8MB/s]


In [1]:
import timm

We have now downloaded and extracted the data to the `data` directory.

In [2]:
ls data

paddy-disease-classification.zip  test_images/  train_images/
sample_submission.csv             train.csv


In [3]:
ls data/train_images

bacterial_leaf_blight/     dead_heart/    vit_small_patch16_224_squishvitsmpad
bacterial_leaf_streak/     downy_mildew/  vit_tiny_patch16_224_squishvittiny
bacterial_panicle_blight/  hispa/         vit_tiny_r_s16_p8_224_squishvitsm
blast/                     normal/
brown_spot/                tungro/


In [4]:
ls data/test_images | head

200001.jpg
200002.jpg
200003.jpg
200004.jpg
200005.jpg
200006.jpg
200007.jpg
200008.jpg
200009.jpg
200010.jpg
ls: write error: Broken pipe


Seems that the train data is organized by directories, with the name of the directory being the label.

Test images just live in `data/test_images`

Let us see what is the format of the sample submission file to have a full picture.

In [5]:
import pandas as pd

sample_sub = pd.read_csv('data/sample_submission.csv')
sample_sub.head()

,image_id,label
0,200001.jpg,NaN
1,200002.jpg,NaN
2,200003.jpg,NaN
3,200004.jpg,NaN
4,200005.jpg,NaN


Mhmm. Guessing the labels for the submission are the names of the directories.

Ok, let's start training!

In [6]:
from fastai.vision.all import *
from fastcore.parallel import *

In [7]:
path = Path('data')
trn_path= path/'train_images'
tst_files = get_image_files(path/'test_images').sorted()

In [10]:
def train(desc, arch, item, batch, accum=False):
    kwargs = {'bs':32} if accum else{}
    dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=item, batch_tfms=batch, **kwargs)
    cbs = GradientAccumulation(2) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    learn.fine_tune(15, 0.02)
    tta_res.append(learn.tta(dl=dls.test_dl(tst_files)))
    learn.export(f'{arch}_{desc}')
    

In [18]:
timm.list_models("convnext_large*")

['convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k']

In [19]:
arch = 'convnext_large_in22k'

In [20]:
train('Convnext', arch, item=Resize(480, method='squish'), batch=aug_transforms(size=224, min_scale=0.75), accum=False)

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_large_22k_224.pth


epoch,train_loss,valid_loss,error_rate,time
0,1.049075,0.656005,0.206631,04:27


epoch,train_loss,valid_loss,error_rate,time
0,0.453522,0.289151,0.090822,05:51
1,0.306500,0.223337,0.070159,05:50
2,0.254169,0.221738,0.067275,05:50
3,0.267119,0.213160,0.065353,05:50
4,0.235032,0.254823,0.067756,05:50
5,0.202015,0.210563,0.055262,05:50
6,0.186112,0.253782,0.061028,05:51
7,0.153320,0.180753,0.044690,05:51
8,0.144047,0.195207,0.044690,05:51
9,0.107127,0.181230,0.039885,05:51


In [ ]:
save_pickle('tta_res_convnext6k.pkl', tta_res)

In [11]:
tta_res=load_pickle('tta_res_convnext6k.pkl')

In [18]:
timm.list_models('vit*', pretrained=True)

['vit_base_patch8_224',
 'vit_base_patch8_224_dino',
 'vit_base_patch8_224_in21k',
 'vit_base_patch16_224',
 'vit_base_patch16_224_dino',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_224_sam',
 'vit_base_patch16_384',
 'vit_base_patch16_rpn_224',
 'vit_base_patch32_224',
 'vit_base_patch32_224_in21k',
 'vit_base_patch32_224_sam',
 'vit_base_patch32_384',
 'vit_base_r50_s16_224_in21k',
 'vit_base_r50_s16_384',
 'vit_huge_patch14_224_in21k',
 'vit_large_patch16_224',
 'vit_large_patch16_224_in21k',
 'vit_large_patch16_384',
 'vit_large_patch32_224_in21k',
 'vit_large_patch32_384',
 'vit_large_r50_s32_224',
 'vit_large_r50_s32_224_in21k',
 'vit_large_r50_s32_384',
 'vit_relpos_base_patch16_224',
 'vit_relpos_base_patch16_gapcls_224',
 'vit_relpos_base_patch32_plus_rpn_256',
 'vit_relpos_medium_patch16_224',
 'vit_relpos_medium_patch16_rpn_224',
 'vit_relpos_small_patch16_224',
 'vit_small_patch8_224_dino',
 'vit_small_

In [12]:
arch = 'vit_base_patch16_224'

I had issues with a 'dataloader' error on the larger models - reducing the resize (not size) avoided this. Maybe try going smaller with a larger vit model.  Or use gradient accumulation. 

In [14]:
train('squishvitbasepad', arch, item=Resize((288,208), method=ResizeMethod.Pad, pad_mode=PadMode.Zeros), batch=aug_transforms(size=224, min_scale=0.75), accum=False)

epoch,train_loss,valid_loss,error_rate,time
0,1.243602,1.077358,0.261413,02:00


epoch,train_loss,valid_loss,error_rate,time
0,0.581251,0.341382,0.113888,02:43
1,0.435181,0.879958,0.187410,02:46
2,0.468518,1.004132,0.283518,02:44
3,0.424309,0.720214,0.198462,02:46
4,0.390805,0.452324,0.147525,02:46
5,0.303213,0.772633,0.176838,02:46
6,0.238583,0.286839,0.080250,02:45
7,0.193392,0.214526,0.056704,02:44
8,0.133780,0.177797,0.044690,02:45
9,0.104493,0.177561,0.047573,02:45


In [15]:
save_pickle('tta_res_post_vitp6base.pkl', tta_res)

In [24]:
tta_res=load_pickle('tta_res_post_vitp6base.pkl')

In [12]:
timm.list_models('swinv2*')

['swinv2_base_window8_256',
 'swinv2_base_window12_192_22k',
 'swinv2_base_window12to16_192to256_22kft1k',
 'swinv2_base_window12to24_192to384_22kft1k',
 'swinv2_base_window16_256',
 'swinv2_cr_base_224',
 'swinv2_cr_base_384',
 'swinv2_cr_base_ns_224',
 'swinv2_cr_giant_224',
 'swinv2_cr_giant_384',
 'swinv2_cr_huge_224',
 'swinv2_cr_huge_384',
 'swinv2_cr_large_224',
 'swinv2_cr_large_384',
 'swinv2_cr_small_224',
 'swinv2_cr_small_384',
 'swinv2_cr_small_ns_224',
 'swinv2_cr_tiny_224',
 'swinv2_cr_tiny_384',
 'swinv2_cr_tiny_ns_224',
 'swinv2_large_window12_192_22k',
 'swinv2_large_window12to16_192to256_22kft1k',
 'swinv2_large_window12to24_192to384_22kft1k',
 'swinv2_small_window8_256',
 'swinv2_small_window16_256',
 'swinv2_tiny_window8_256',
 'swinv2_tiny_window16_256']

In [25]:
arch = 'swinv2_large_window12_192_22k'

In [26]:
train('squishswin2192', arch, item=Resize(240, method='squish'), batch=aug_transforms(size=192, min_scale=0.75), accum=True)

/opt/conda/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272168290/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v2.0.0/swinv2_large_patch4_window12_192_22k.pth" to /root/.cache/torch/hub/checkpoints/swinv2_large_patch4_window12_192_22k.pth


epoch,train_loss,valid_loss,error_rate,time
0,1.313985,0.813090,0.233061,04:33


epoch,train_loss,valid_loss,error_rate,time
0,0.550304,0.289760,0.088419,06:00
1,0.443680,0.329001,0.098030,05:59
2,0.468659,0.289688,0.092744,05:59
3,0.431333,0.411514,0.136953,05:58
4,0.341946,0.347437,0.092263,05:59
5,0.307648,0.276687,0.081691,05:59
6,0.258507,0.191180,0.047093,05:58
7,0.170284,0.182750,0.046132,05:59
8,0.125255,0.157833,0.038924,05:59
9,0.084183,0.164015,0.030274,05:59


In [27]:
save_pickle('tta_res_post_swin2p6lg.pkl', tta_res)
# tta_prs = first(zip(*tta_res))

In [46]:
tta_res=load_pickle('tta_res_post_swin2p6.pkl')

In [28]:
tta_prs = first(zip(*tta_res))

The following cells can be used if any of the models deserves a higher representation in the ensemble - by duplicating their contribution before averaging.

In [72]:
#tta_prs += tta_prs[0:1]

In [61]:
#tta_prs += tta_prs[1:2]

In [19]:
#tta_prs += tta_prs[3:4]

In [30]:
t_tta = torch.stack(tta_prs)

In [31]:
avg_pr = t_tta.mean(0)
idxs = avg_pr.argmax(dim=1)
idxs.shape

torch.Size([3469])

In [21]:
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=Resize(224))
mapping = dict(enumerate(dls.vocab))
ss = pd.read_csv('data/sample_submission.csv')
results = pd.Series(idxs.numpy(), name='idxs').map(mapping)
ss.label = results
ss.to_csv('data/submissions/subm.csv', index=False)

Check out the submission just to see if it looks ok format wise

In [22]:
ss

,image_id,label
0,200001.jpg,hispa
1,200002.jpg,normal
2,200003.jpg,blast
3,200004.jpg,blast
4,200005.jpg,blast
...,...,...
3464,203465.jpg,dead_heart
3465,203466.jpg,hispa
3466,203467.jpg,normal
3467,203468.jpg,bacterial_leaf_streak


In [23]:
!kaggle competitions submit -c paddy-disease-classification -f data/submissions/subm.csv -m "13th Mix conv vit"

100%|███████████████████████████████████████| 70.5k/70.5k [00:00<00:00, 139kB/s]
Successfully submitted to Paddy Doctor: Paddy Disease Classification